<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/SCM_275x_Decisions_on_Demand_to_Serve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Decisions on Demand to Serve**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points


## **Helper functions**

### **Ploting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               active_color,                # Color of the marker icon for active nodes
               background_color,            # Background color of the marker icon
               inactive_color = 'grey',     # Color of the marker icon for inactive nodes
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=active_color if node.active == True else inactive_color,
                border_color=active_color if node.active == True else inactive_color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi')

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Ploting flows of different products on a map**

In [ ]:
# Defining a function to plot flows on a map using folium

def plot_flows_products(map,          # Folium map object where flows will be plotted.
               vars,                  # Dictionary of decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               product_colors,        # Dictionary representing colors of the lines representing flows of different products
               max_width = 30,        # Maximum line width for the flows, default is 30
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key, product), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key, product), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

          # Get the coordinates of the nodes for plotting the line
          points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


          # Add a PolyLine to the map to represent the flow between the nodes
          folium.PolyLine(points,
                            color=product_colors[product],              # Set the color of the line
                            weight=var.X / max_val * max_width,         # Normalize line width based on flow value
                            opacity=opacity,                            # Set line opacity
                            popup=var.X).add_to(map)                    # Show the flow value in a popup on the map

## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/demand_decisions/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)

# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,lat,lon
0,c1,Charlotte,35.2083,-80.8303
1,c2,Milwaukee,43.0642,-87.9675
2,c3,Providence,41.8230,-71.4187
3,c4,Jacksonville,30.3322,-81.6749
4,c5,Nashville,36.1715,-86.7842


In [ ]:
# File containing demand data
demand_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/demand_decisions/demand_data.csv'

# Loading demand data file into a pandas DataFrame
demand_data_df = pd.read_csv(demand_data_file)

# Displaying the first few rows of the DataFrame to verify the data
demand_data_df.head()

,customerID,product,demand,price
0,c1,product_a,14300,50
1,c2,product_a,12900,50
2,c3,product_a,12900,50
3,c4,product_a,12900,50
4,c5,product_a,11800,50


In [ ]:
# File containing warehouse data
warehouse_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/demand_decisions/warehouses.csv'

# Loading warehouse data into a pandas DataFrame
warehouses_df = pd.read_csv(warehouse_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehouses_df.head()

,ID,name,lat,lon,capacity,fixed_cost
0,w1,Detroit,42.3834,-83.1024,100000,1000000
1,w2,Denver,39.7620,-104.8758,100000,1000000
2,w3,Portland,45.5371,-122.6500,100000,1000000
3,w4,Minneapolis,44.9635,-93.2678,100000,1000000
4,w5,Orlando,28.4773,-81.3370,100000,1000000


In [ ]:
# File containing landed cost data
warehousing_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/demand_decisions/warehousing_data.csv'

# Loading landed cost data into a pandas DataFrame
warehousing_df = pd.read_csv(warehousing_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehousing_df.head()

,warehouseID,productID,landed_cost
0,w1,product_a,40
1,w1,product_b,56
2,w2,product_a,41
3,w2,product_b,58
4,w3,product_a,39


#### Definition of Classes

In [ ]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon):
        self.ID = ID                    # Customer's ID
        self.name = name                # Customer's name
        self.lat = lat                  # Customer's latitude
        self.lon = lon                  # Customer's longitude

        self.demand = dict()            # Initializing an empty dictionnary to store customers's demand per product
        self.selling_price = dict()     # Initializing an empty dictionnary to store customers's demand per product

        self.active = True              # Initializing node as active


In [ ]:
# Class representing a Warehouse object

class Warehouse():
    def __init__(self, ID, name, lat, lon, fixed_cost, capacity):
        self.ID = ID                  # Warehouse's ID
        self.name = name              # Warehouse's name
        self.lat = lat                # Warehouse's latitude
        self.lon = lon                # Warehouse's longitude
        self.fixed_cost = fixed_cost  # Warehouse's fixed cost
        self.capacity = capacity      # Warehouse's capacity

        self.landed_cost = dict()     # Initializing an empty dictionnary to store warehouse's landed cost per product

        self.active = True            # Initializing node as active


#### Creating node objects

In [ ]:
# Initializing an empty dictionary to store node objects
nodes = dict()

In [ ]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'])         # Customer's longitude

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [ ]:
# Associating demand data to customers
for i, row in demand_data_df.iterrows():
    customers[row['customerID']].demand[row['product']] = row['demand']
    customers[row['customerID']].selling_price[row['product']] = row['price']

In [ ]:
# Visualizing an example of customer demand
customers['c1'].demand

{'product_a': 14300, 'product_b': 21450}

In [ ]:
# Visualizing an example of customer selling price
customers['c1'].selling_price

{'product_a': 50, 'product_b': 90}

In [ ]:
# Creating a dictionary of warehouse objects
warehouses = dict()
for i, row in warehouses_df.iterrows():
    warehouses[row['ID']] = Warehouse(ID = row['ID'],               # Warehouse's ID
                                    name = row['name'],             # Warehouse's name
                                    lat = row['lat'],               # Warehouse's latitude
                                    lon = row['lon'],               # Warehouse's longitude
                                    fixed_cost = row['fixed_cost'], # Warehouse's fixed cost
                                    capacity = row['capacity'])     # Warehouse's fixed cost

# Merging the warehouse dictionary into the existing nodes dictionary
nodes = {**nodes, **warehouses}

In [ ]:
# Associating warehousing data to warehouses
for i, row in warehousing_df.iterrows():
    warehouses[row['warehouseID']].landed_cost[row['productID']] = row['landed_cost']

#### Visualizing node objects

In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary containing distances between warehouses and customers

distances = dict()
for w, warehouse in warehouses.items():
  for c, customer in customers.items():
      distances[w, c] = compute_geodesic_distance(origin = warehouse, destination = customer, unit = 'km')

#### Arc costs

In [ ]:
cost_unit_km_warehouse_customer = 0.01    # Cost per unit per kilometer from warehouse to customer

# Creating a dictionary containing unit costs between suppliers and warehouses, and between warehouses and customers
unit_cost = dict()

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        unit_cost[w, c] = distances[w, c] * cost_unit_km_warehouse_customer                    # Calculate unit cost as distance multiplied by cost per km (warehouse to customer)


### **Products**

In [ ]:
# Defining a list of products to consider in the model
products = ['product_a', 'product_b']

## **Optimization model - Cost Minimization**

### **Creating and solving the optimization model**

In [ ]:
# Initializing the optimization model for cost minimization
model = grb.Model("Cost minimization model")

# Creating binary decision variables to determine if a warehouse is open
wh_location_vars = dict()
for w, warehouse in warehouses.items():  # Iterate over warehouses
    wh_location_vars[w] = model.addVar(vtype=grb.GRB.BINARY,
                               name = "wh_location_{0}".format(w))

# Creating continuous decision variables for flow from warehouses to customers
flow_vars = dict()
for w, warehouse in warehouses.items():  # Iterate over warehouses
    for c, customer in customers.items():  # Iterate over customers
        for p in products:  # Iterate over products
            flow_vars[w, c, p] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_{0}_{1}_{2}".format(w, c, p))

# Creating the objective function

# Calculating fixed costs for opening warehouses
fixed_cost_warehouses = grb.quicksum(wh_location_vars[w]
                          * 1.5 * warehouse.fixed_cost
                          for w, warehouse in warehouses.items())

# Calculating transportation costs for flows from warehouses to customers
cost_warehouse_customer = grb.quicksum(flow_vars[w, c, p] * unit_cost[w, c]
                        for w, warehouse in warehouses.items()
                        for c, customer in customers.items()
                        for p in products)

# Calculating landed costs for products in each warehouse
landed_cost = grb.quicksum(flow_vars[w, c, p] * warehouse.landed_cost[p]
                        for w, warehouse in warehouses.items()
                        for c, customer in customers.items()
                        for p in products)

# Total cost is the sum of fixed warehouse costs and transportation costs
total_cost = fixed_cost_warehouses + cost_warehouse_customer + landed_cost

# Setting the objective to minimize the total cost
model.setObjective(total_cost, grb.GRB.MINIMIZE)

# Adding demand constraints (flow into each customer must meet their demand)
for c, customer in customers.items():
    for p in products:
        model.addConstr(grb.quicksum(flow_vars[w, c, p] for w, warehouse in warehouses.items()) == customer.demand[p])

# Adding warehouse capacity constraints (flow out of each warehouse must not exceed its capacity)
for w, warehouse in warehouses.items():
    model.addConstr(grb.quicksum(flow_vars[w, c, p] for c, customer in customers.items() for p in products) <= wh_location_vars[w] * 1.5 * warehouse.capacity)

# Solving the model
model.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 75 rows, 355 columns and 705 nonzeros
Model fingerprint: 0xc8396bb2
Variable types: 350 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [4e+01, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 2e+04]
Presolve time: 0.00s
Presolved: 75 rows, 355 columns, 705 nonzeros
Variable types: 350 continuous, 5 integer (5 binary)
Found heuristic solution: objective 4.273373e+07

Root relaxation: objective 4.147623e+07, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.1476e+07    0    2 4.2734e+07 4.1476e+

In [ ]:
# Calculate and print the total capacity across all warehouses
total_warehouse_capacity = 0
for warehouse in warehouses.values():
    total_warehouse_capacity += warehouse.capacity

# Calculate and print the total demand for both 'product_a' and 'product_b' across all customers
demand_product_a = 0
demand_product_b = 0
for customer in customers.values():
  demand_product_a += customer.demand['product_a']
  demand_product_b += customer.demand['product_b']

print('Total warehouse capacity: ', total_warehouse_capacity)

print ('--')
print('Total demand product A: ', demand_product_a)
print('Total demand product B: ', demand_product_b)
print('Total demand: ', demand_product_a + demand_product_b)

Total warehouse capacity:  500000
--
Total demand product A:  312400
Total demand product B:  311850
Total demand:  624250


### **Solution visualization and analysis**

In [ ]:
print('Fixed Cost Warehouses: ', fixed_cost_warehouses.getValue())
print('Transportation Cost Warehouse to Customer: ', cost_warehouse_customer.getValue())

Fixed Cost Warehouses:  7500000.0
Transportation Cost Warehouse to Customer:  4923131.851709988


In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Define different colors for plotting products
colors = {'product_a': 'red',
          'product_b': 'green'}

# Plot flows on the map
plot_flows_products(map = map, vars = flow_vars, nodes = nodes, product_colors = colors, opacity = 0.3)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


## **Optimization model - Profit Maximization**

### **Creating and solving the optimization model**

In [ ]:
# Initializing the optimization model for profit maximization
model = grb.Model("Profit maximization model")

# Creating binary decision variables to determine if a warehouse is open
wh_location_vars = dict()
for w, warehouse in warehouses.items():  # Iterate over warehouses
    wh_location_vars[w] = model.addVar(vtype=grb.GRB.BINARY,
                               name = "wh_location_{0}".format(w))

# Creating continuous decision variables for flow from warehouses to customers
flow_vars = dict()
for w, warehouse in warehouses.items():  # Iterate over warehouses
    for c, customer in customers.items():  # Iterate over customers
        for p in products:  # Iterate over products
            flow_vars[w, c, p] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_{0}_{1}_{2}".format(w, c, p))

# Creating the objective function

# Calculating revenue from selling products to customers
revenue = grb.quicksum(flow_vars[w, c, p] * cust.selling_price[p]
                        for w, warehouse in warehouses.items()
                        for c, cust in customers.items()
                        for p in products)

# Calculating fixed costs for opening warehouses
fixed_cost_warehouses = grb.quicksum(wh_location_vars[w]
                          * warehouse.fixed_cost
                          for w, warehouse in warehouses.items())

# Calculating transportation costs for flows from warehouses to customers
cost_warehouse_customer = grb.quicksum(flow_vars[w, c, p] * unit_cost[w, c]
                        for w, warehouse in warehouses.items()
                        for c, customer in customers.items()
                        for p in products)

# Calculating landed costs for products in each warehouse
landed_cost = grb.quicksum(flow_vars[w, c, p] * warehouse.landed_cost[p]
                        for w, warehouse in warehouses.items()
                        for c, customer in customers.items()
                        for p in products)

# Total profit is revenue minus fixed costs, transportation costs, and landed costs
total_profit = revenue - fixed_cost_warehouses - cost_warehouse_customer - landed_cost

# Setting the objective to maximize total profit
model.setObjective(total_profit, grb.GRB.MAXIMIZE)

# Adding demand constraints (flow to each customer must not exceed their demand)
for c, customer in customers.items():
    for p in products:
        model.addConstr(grb.quicksum(flow_vars[w, c, p] for w, warehouse in warehouses.items()) <= customer.demand[p])

# Adding warehouse capacity constraints (flow out of each warehouse must not exceed its capacity)
for w, warehouse in warehouses.items():
    model.addConstr(grb.quicksum(flow_vars[w, c, p] for c, customer in customers.items() for p in products) <= wh_location_vars[w] * warehouse.capacity)

# Solving the optimization model
model.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 75 rows, 355 columns and 705 nonzeros
Model fingerprint: 0x1464c872
Variable types: 350 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 2e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 19 rows and 127 columns
Presolve time: 0.00s
Presolved: 56 rows, 228 columns, 436 nonzeros
Variable types: 223 continuous, 5 integer (5 binary)

Root relaxation: objective 8.328344e+06, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8

### **Solution visualization and analysis**

In [ ]:
print('Fixed Cost Warehouses: ', fixed_cost_warehouses.getValue())
print('Transportation Cost Warehouse to Customer: ', cost_warehouse_customer.getValue())


Fixed Cost Warehouses:  4000000.0
Transportation Cost Warehouse to Customer:  2987565.1741872416


In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Define different colors for plotting products
colors = {'product_a': 'red',
          'product_b': 'green'}

# Plot flows on the map
plot_flows_products(map = map, vars = flow_vars, nodes = nodes, product_colors = colors, opacity = 0.3)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


In [ ]:
# Initialize variables to track the total demand served for product_a and product_b
demand_product_a_served = 0
demand_product_b_served = 0

# Iterate over all warehouses and customers
for w, warehouse in warehouses.items():
    for c, customer in customers.items():
        # Add the flow of product_a from warehouse w to customer c to the total demand served for product_a
        demand_product_a_served += flow_vars[w, c, 'product_a'].X
        # Add the flow of product_b from warehouse w to customer c to the total demand served for product_b
        demand_product_b_served += flow_vars[w, c, 'product_b'].X

# Print the total demand served for product_a
print('Demand product_a served: ', demand_product_a_served)
# Print the total demand served for product_b
print('Demand product_b served: ', demand_product_b_served)
# Print the total demand served for both products
print('Total demand served: ', demand_product_a_served + demand_product_b_served)

# Separator for output readability
print('---')

# Print the total capacity of all warehouses
print('Total warehouse capacity', total_warehouse_capacity)


Demand product_a served:  88150.0
Demand product_b served:  311850.0
Total demand served:  400000.0
---
Total warehouse capacity 500000


In [ ]:
# Calculate and print the percentage of demand for Product A that has been served
print('Percentage of demand of Product A served: ', round(100 * demand_product_a_served / demand_product_a, 2))

# Calculate and print the percentage of demand for Product B that has been served
print('Percentage of demand of Product B served: ', round(100 * demand_product_b_served / demand_product_b, 2))


Percentage of demand of Product A served:  28.22
Percentage of demand of Product B served:  100.0


In [ ]:
# Iterate over each customer in the customers dictionary
for c, customer in customers.items():
    # Calculate the total demand for Product A served to the current customer
    demand_product_a_per_customer = grb.quicksum(flow_vars[w, c, 'product_a'] for w, warehouse in warehouses.items()).getValue()

    # Print the customer ID and the percentage of their demand for Product A that has been served
    print(customer.ID, round(100 * demand_product_a_per_customer / customer.demand['product_a'], 2))

c1 0.0
c2 0.0
c3 0.0
c4 100.0
c5 0.0
c6 0.0
c7 0.0
c8 100.0
c9 0.0
c10 100.0
c11 0.0
c12 0.0
c13 100.0
c14 0.0
c15 0.0
c16 0.0
c17 0.0
c18 0.0
c19 56.71
c20 0.0
c21 0.0
c22 0.0
c23 100.0
c24 100.0
c25 0.0
c26 0.0
c27 0.0
c28 0.0
c29 100.0
c30 0.0
c31 100.0
c32 100.0
c33 0.0
c34 0.0
c35 100.0


In [ ]:
# Iterate over each customer in the customers dictionary
for c, customer in customers.items():
    # Print the customer ID and the selling price for Product A
    print(customer.ID, customer.selling_price['product_a'])

c1 50
c2 50
c3 50
c4 50
c5 50
c6 50
c7 50
c8 60
c9 50
c10 60
c11 50
c12 50
c13 60
c14 50
c15 50
c16 50
c17 50
c18 50
c19 50
c20 50
c21 50
c22 50
c23 60
c24 70
c25 50
c26 50
c27 50
c28 50
c29 60
c30 50
c31 60
c32 60
c33 50
c34 50
c35 60


In [ ]:
# Iterate over each warehouse in the warehouses dictionary
for w, warehouse in warehouses.items():
    # Calculate the total outgoing flows from the current warehouse
    outgoing_flows = grb.quicksum(flow_vars[w, c, 'product_a'] + flow_vars[w, c, 'product_b'] for c, customer in customers.items()).getValue()

    # Print the warehouse ID and the percentage of the warehouse's capacity that is being used
    print(warehouse.ID, round(100 * outgoing_flows / warehouse.capacity, 2))


w1 100.0
w2 100.0
w3 0.0
w4 100.0
w5 100.0
